In [1]:
import pandas as pd
from tensorflow import keras
from tensorflow.keras import layers
from keras_tuner.tuners import RandomSearch

2022-12-21 21:05:06.182281: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


## Predict Air Quality Index prediction

In [2]:
df = pd.read_csv('Real_Combine.csv')

In [3]:
df.head()

,T,TM,Tm,SLP,H,VV,V,VM,PM 2.5
0,7.4,9.8,4.8,1017.6,93.0,0.5,4.3,9.4,219.720833
1,7.8,12.7,4.4,1018.5,87.0,0.6,4.4,11.1,182.187500
2,6.7,13.4,2.4,1019.4,82.0,0.6,4.8,11.1,154.037500
3,8.6,15.5,3.3,1018.7,72.0,0.8,8.1,20.6,223.208333
4,12.4,20.9,4.4,1017.3,61.0,1.3,8.7,22.2,200.645833


In [4]:
X=df.iloc[:,:-1]
y=df.iloc[:,-1]

#### Hyperparameters
1. How many number of hidden layers we should have?
2. How many neurons we should have in hidden layer?
3. What should be the Learning Rate

In [5]:
def build_model(hp):
    model = keras.Sequential()
    for i in range(hp.Int('num_layers', 2, 20)):
        model.add(layers.Dense(units=hp.Int('units_' + str(i), min_value=32, max_value=512, step=32),activation='relu'))
    model.add(layers.Dense(units=1, activation='linear'))
    model.compile(
        optimizer=keras.optimizers.Adam(
            hp.Choice('learning_rate', [1e-2, 1e-3, 1e-4])),
        loss='mean_absolute_error',
        metrics=['mean_absolute_error'])
    return model

In [6]:
tuner = RandomSearch(
    build_model,
    objective='val_mean_absolute_error',
    max_trials=5,
    executions_per_trial=3,
    directory='project',
    overwrite=True,
    project_name='Air Quality Index')

2022-12-21 21:05:12.027447: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [7]:
tuner.search_space_summary()

Search space summary
Default search space size: 4
num_layers (Int)
{'default': None, 'conditions': [], 'min_value': 2, 'max_value': 20, 'step': 1, 'sampling': None}
units_0 (Int)
{'default': None, 'conditions': [], 'min_value': 32, 'max_value': 512, 'step': 32, 'sampling': None}
units_1 (Int)
{'default': None, 'conditions': [], 'min_value': 32, 'max_value': 512, 'step': 32, 'sampling': None}
learning_rate (Choice)
{'default': 0.01, 'conditions': [], 'values': [0.01, 0.001, 0.0001], 'ordered': True}


In [8]:
from sklearn.model_selection import train_test_split
X_train, X_test,y_train, y_test = train_test_split(X, y, test_size=0.3,random_state=0)

In [9]:
tuner.search(X_train, y_train, epochs=5, validation_data=(X_test, y_test))

Trial 5 Complete [00h 00m 05s]
val_mean_absolute_error: 64.9302469889323

Best val_mean_absolute_error So Far: 52.4703369140625
Total elapsed time: 00h 00m 56s
INFO:tensorflow:Oracle triggered exit


In [10]:
tuner.results_summary()

Results summary
Results in project/Air Quality Index
Showing 10 best trials
Trial summary
Hyperparameters:
num_layers: 16
units_0: 224
units_1: 160
learning_rate: 0.01
units_2: 32
units_3: 32
units_4: 32
units_5: 32
units_6: 32
units_7: 32
units_8: 32
units_9: 32
units_10: 32
units_11: 32
units_12: 32
units_13: 32
units_14: 32
units_15: 32
Score: 52.4703369140625
Trial summary
Hyperparameters:
num_layers: 7
units_0: 192
units_1: 320
learning_rate: 0.001
units_2: 256
units_3: 96
units_4: 96
units_5: 320
units_6: 352
units_7: 192
units_8: 448
units_9: 384
units_10: 320
units_11: 32
units_12: 288
units_13: 320
units_14: 32
units_15: 32
units_16: 352
units_17: 160
units_18: 64
units_19: 480
Score: 62.663954416910805
Trial summary
Hyperparameters:
num_layers: 20
units_0: 192
units_1: 416
learning_rate: 0.001
units_2: 160
units_3: 160
units_4: 192
units_5: 384
units_6: 384
units_7: 448
units_8: 32
units_9: 352
units_10: 96
units_11: 64
units_12: 448
units_13: 192
units_14: 416
units_15: 256
